In [138]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

## lấy mfcc của file wav

In [139]:
# read file
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

## lấy mfcc của tất cả các file trong dir

In [140]:
# lấy mfcc của tất cả các file wav trong wav
def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

## Hàm Clustering

In [141]:
def clustering(X, n_clusters=14):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans

## Data

In [142]:
class_names = ["khong","tôi","cachly", "chungta" , 'nguoi'  ]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join( cname))


Load khong dataset
Load tôi dataset
Load cachly dataset
Load chungta dataset
Load nguoi dataset


In [143]:
print(len(dataset['khong']))

99


In [144]:
trainset = {}
testset = {}
n_test = {'khong': 0, 'tôi': 0, 'cachly': 0, 'chungta': 0,'nguoi': 0}
for cname in class_names:
    n = len(dataset[cname])
    n_train = math.floor(n*0.7)
    trainset[cname] = dataset[cname][:n_train]
    testset[cname] = dataset[cname][n_train:]
    n_test[cname] += len(testset[cname])
    
print(len(trainset['khong']))

69


## Fit kmeans trên tập train

In [145]:
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in trainset.items()], axis=0)
print("vectors", all_vectors.shape)
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors)
print("centers", kmeans.cluster_centers_.shape)
print(kmeans)

vectors (14828, 36)
centers (14, 36)
centers (14, 36)
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=14, n_init=50, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)


In [146]:
for cname in class_names:
    trainset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in trainset[cname]])
    testset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in testset[cname]])

    

# Train

In [147]:
models = {}

## Model cho 'trong' 3x3


In [148]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=3*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.6,0.3,0.1,0.0,0.0,0.0,0.0,0.0,0.0])
hmm.transmat_ = np.array([
        [0.5,0.3,0.2,0.0,0.0,0.0,0.0,0.0,0.0],
        [0.0,0.5,0.3,0.2,0.0,0.0,0.0,0.0,0.0],
        [0.0,0.0,0.5,0.3,0.2,0.0,0.0,0.0,0.0],
        [0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0,0.0],
        [0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.4],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0],
    ])
Xbn = np.concatenate(trainset['tôi'])
lengths = list([len(x) for x in trainset['tôi']])
print("training class", 'tôi')
print(Xbn.shape, lengths, len(lengths))
hmm.fit(Xbn, lengths=lengths)
models['tôi'] = hmm
print("Training done")

training class tôi
(752, 1) [15, 17, 11, 13, 17, 21, 21, 14, 15, 12, 11, 18, 15, 16, 10, 27, 42, 17, 17, 30, 30, 12, 14, 14, 14, 18, 25, 17, 15, 9, 17, 22, 12, 12, 30, 36, 17, 15, 13, 15, 16, 20] 42


         1       -1947.3763             +nan
         2       -1316.8090        +630.5673
         3       -1168.2716        +148.5374
         4       -1108.9502         +59.3214
         5       -1077.3403         +31.6099
         6       -1062.1979         +15.1424
         7       -1053.7554          +8.4424
         8       -1046.3397          +7.4157
         9       -1039.5296          +6.8101
        10       -1033.5266          +6.0030
        11       -1028.0082          +5.5185
        12       -1023.1567          +4.8515
        13       -1019.3966          +3.7600
        14       -1016.5969          +2.7997
        15       -1014.0818          +2.5151
        16       -1011.9334          +2.1485
        17       -1010.1708          +1.7626
        18       -1007.9262          +2.2446
        19       -1006.0843          +1.8419
        20       -1005.0148          +1.0695
        21       -1004.0909          +0.9239
        22       -1003.2726          +0.8182
        23

Training done


## Model cho 'không' 3x3

In [149]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=3*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xct = np.concatenate(trainset['khong'])
lengths = list([len(x) for x in trainset['khong']])
print("training class", 'khong')
print(Xct.shape, lengths, len(lengths))
hmm.fit(Xct, lengths=lengths)
models['khong'] = hmm
print("Training done")

         1       -9923.9092             +nan
         2       -3805.4575       +6118.4517
         3       -3221.1122        +584.3453


training class khong
(3943, 1) [26, 101, 29, 101, 31, 34, 25, 15, 22, 46, 30, 33, 101, 25, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 38, 19, 27, 22, 31, 25, 31, 26, 27, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 23, 25, 23, 45, 26, 19, 23, 30, 24, 101, 28, 28, 27, 25, 31, 32, 23, 22, 18, 28, 101, 29, 24] 69


         4       -2972.0441        +249.0681
         5       -2590.1777        +381.8664
         6       -2523.5079         +66.6698
         7       -2502.0478         +21.4601
         8       -2498.0873          +3.9605
         9       -2497.2987          +0.7886
        10       -2497.0403          +0.2584
        11       -2496.8540          +0.1863
        12       -2496.6424          +0.2116
        13       -2496.4072          +0.2352
        14       -2496.2024          +0.2048
        15       -2496.0646          +0.1378
        16       -2495.9844          +0.0802
        17       -2495.9380          +0.0464
        18       -2495.9090          +0.0291
        19       -2495.8889          +0.0201
        20       -2495.8739          +0.0150
        21       -2495.8622          +0.0117


Training done


        22       -2495.8528          +0.0094


## Model cho 'chúng ta' 5x3

In [150]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=5*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xct = np.concatenate(trainset['chungta'])
lengths = list([len(x) for x in trainset['chungta']])
print("training class", 'chungta')
print(Xct.shape, lengths, len(lengths))
hmm.fit(Xct, lengths=lengths)
models['chungta'] = hmm
print("Training done")

training class chungta
(2759, 1) [41, 38, 28, 46, 47, 36, 43, 34, 36, 42, 36, 40, 36, 37, 30, 40, 36, 41, 36, 37, 36, 42, 40, 35, 39, 38, 38, 41, 36, 44, 38, 59, 49, 37, 31, 43, 52, 42, 50, 48, 46, 54, 55, 47, 45, 35, 38, 38, 42, 38, 43, 33, 35, 32, 42, 32, 39, 41, 40, 37, 35, 37, 47, 31, 34, 36, 34, 34, 31, 30] 70


         1       -7277.2052             +nan
         2       -4430.2630       +2846.9422
         3       -3691.6995        +738.5635
         4       -3376.0417        +315.6578
         5       -3254.6046        +121.4371
         6       -3202.1050         +52.4995
         7       -3167.4954         +34.6096
         8       -3144.2036         +23.2919
         9       -3133.1564         +11.0471
        10       -3126.8390          +6.3174
        11       -3121.3974          +5.4416
        12       -3115.5329          +5.8645
        13       -3110.5442          +4.9887
        14       -3107.3638          +3.1804
        15       -3105.5228          +1.8410
        16       -3104.3796          +1.1431
        17       -3103.5712          +0.8084
        18       -3102.9006          +0.6707
        19       -3102.2351          +0.6654
        20       -3101.4388          +0.7963
        21       -3100.4174          +1.0214
        22       -3099.3323          +1.0851
        23

Training done


        63       -3075.6679          +0.0093


## Model cho 'người' 3x3

In [151]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=3*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xcth = np.concatenate(trainset['nguoi'])
lengths = list([len(x) for x in trainset['nguoi']])
print("training class", 'nguoi')
print(Xcth.shape, lengths, len(lengths))
hmm.fit(Xcth, lengths=lengths)
models['nguoi'] = hmm
print("Training done")

         1       -4331.5619             +nan
         2       -2637.6080       +1693.9539
         3       -2358.1194        +279.4886
         4       -2249.8806        +108.2387
         5       -2184.6397         +65.2409


training class nguoi
(1673, 1) [25, 22, 20, 18, 23, 25, 25, 38, 20, 25, 27, 22, 23, 23, 20, 26, 23, 19, 28, 24, 25, 19, 22, 28, 19, 24, 28, 20, 20, 32, 20, 18, 20, 31, 22, 19, 24, 14, 22, 15, 13, 22, 19, 25, 18, 27, 20, 20, 24, 17, 22, 19, 22, 26, 20, 23, 35, 28, 22, 30, 25, 27, 26, 23, 31, 30, 21, 19, 18, 20, 22, 15, 26] 73


         6       -2133.8010         +50.8387
         7       -2096.1157         +37.6853
         8       -2082.0622         +14.0536
         9       -2074.5602          +7.5020
        10       -2069.7656          +4.7946
        11       -2066.1168          +3.6488
        12       -2063.0868          +3.0300
        13       -2060.4910          +2.5958
        14       -2058.2566          +2.2344
        15       -2056.5507          +1.7059
        16       -2055.2940          +1.2567
        17       -2054.1703          +1.1237
        18       -2052.8300          +1.3403
        19       -2050.8995          +1.9306
        20       -2048.6746          +2.2248
        21       -2046.8430          +1.8316
        22       -2045.3426          +1.5004
        23       -2043.8057          +1.5370
        24       -2041.8606          +1.9451
        25       -2039.3092          +2.5514
        26       -2036.0760          +3.2332
        27       -2032.2723          +3.8037
        28

Training done


        65       -1956.8393          +0.0118
        66       -1956.8297          +0.0096


## Model cho 'Cách Ly' 5x3

In [152]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=5*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7,0.2,0.1,0.0,0.0,0.0, 0.0,0.0,0.0,0.0,0.0,0.0,0.0, 0.0,0.0])
hmm.transmat_ =np.array([
    [0.5,0.3,0.2,0.0,0.0,0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0,0.0, 0.0,0.0],
        [0.0,0.5,0.3,0.2,0.0,0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.5,0.3,0.2,0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.5,0.3,0.2,0.0, 0.0, 0.0,0.0, 0.0, 0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0,0.0, 0.0, 0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0, 0.0, 0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0, 0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0, 0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2,0.0],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.2],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5],
        [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0],])
Xvn = np.concatenate(trainset['cachly'])
lengths = list([len(x) for x in trainset['cachly']])
print("training class", 'cachly')
print(Xvn.shape, lengths, len(lengths))
hmm.fit(Xvn, lengths=lengths)
models['cachly'] = hmm
print("Training done")

         1      -14739.7238             +nan


training class cachly
(5701, 1) [50, 37, 55, 51, 28, 49, 49, 49, 49, 61, 61, 65, 71, 37, 34, 89, 54, 54, 54, 49, 56, 39, 31, 51, 50, 63, 32, 82, 29, 36, 54, 51, 57, 62, 56, 38, 60, 60, 60, 60, 2957, 51, 59, 61, 89, 64, 64, 64, 64, 39, 58, 58] 52


         2      -11084.0167       +3655.7071
         3      -10405.5979        +678.4188
         4      -10070.4033        +335.1946
         5       -9751.5643        +318.8389
         6       -9508.3696        +243.1947
         7       -9403.8375        +104.5321
         8       -9362.6565         +41.1810
         9       -9343.6440         +19.0125
        10       -9334.6185          +9.0255
        11       -9329.7750          +4.8434
        12       -9326.1812          +3.5938
        13       -9322.3804          +3.8007
        14       -9317.0641          +5.3163
        15       -9308.1906          +8.8735
        16       -9293.2758         +14.9148
        17       -9272.1210         +21.1548
        18       -9247.3870         +24.7340
        19       -9227.4523         +19.9347
        20       -9219.0810          +8.3713
        21       -9216.3229          +2.7582
        22       -9214.2009          +2.1220
        23       -9211.8362          +2.3647
        24

Training done


        48       -9184.7112          +0.0072


# Test

In [153]:
print("Testing")
n_correct = {'khong': 0, 'tôi': 0, 'chungta': 0, 'nguoi': 0,'cachly': 0}
for true_cname in class_names:
    for O in testset[true_cname]:
        score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
        if (true_cname == max(score, key=score.get)): n_correct[true_cname] += 1
        print(true_cname, score, 'predict:', max(score, key=score.get))

Testing
khong {'tôi': -41.15884638829622, 'khong': -31.181263272061482, 'chungta': -38.55125973610246, 'nguoi': -43.54860743681232, 'cachly': -43.917562164662264} predict: khong
khong {'tôi': -85.5174917658584, 'khong': -36.378739221159506, 'chungta': -56.499844130510745, 'nguoi': -53.168727082788244, 'cachly': -58.55225647259785} predict: khong
khong {'tôi': -127.42077440221723, 'khong': -45.64086844465595, 'chungta': -65.00611281393837, 'nguoi': -46.16178887787726, 'cachly': -37.63695729147794} predict: cachly
khong {'tôi': -61.20495462430816, 'khong': -38.505871120256266, 'chungta': -210.04138635265156, 'nguoi': -44.80290930751864, 'cachly': -34.25026909110338} predict: cachly
khong {'tôi': -231.98311818286652, 'khong': -83.30870378810774, 'chungta': -431.1600708774199, 'nguoi': -506.0550095817847, 'cachly': -95.7073196351781} predict: khong
khong {'tôi': -684.9235322899567, 'khong': -45.31264048735336, 'chungta': -458.850902840716, 'nguoi': -62.63654407258419, 'cachly': -51.9275067

In [154]:
for cname in class_names:
    print('Accuracy:', cname, n_correct[cname]/n_test[cname])

Accuracy: khong 0.8666666666666667
Accuracy: tôi 0.6842105263157895
Accuracy: cachly 0.9565217391304348
Accuracy: chungta 0.9333333333333333
Accuracy: nguoi 0.90625


In [155]:
print('All Accuracy:', sum(n_correct.values())/sum(n_test.values()))

All Accuracy: 0.8805970149253731


In [156]:
print(n_test)

{'khong': 30, 'tôi': 19, 'cachly': 23, 'chungta': 30, 'nguoi': 32}


In [157]:
np.around(models['khong'].transmat_, 2)

array([[0.81, 0.19, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.85, 0.  , 0.15, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.8 , 0.01, 0.19, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.84, 0.  , 0.16, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.95, 0.  , 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.97, 0.01, 0.02, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.54, 0.02, 0.44],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

In [158]:
np.around(models['tôi'].transmat_, 2)

array([[0.66, 0.  , 0.34, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.62, 0.01, 0.37, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.09, 0.59, 0.32, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.52, 0.  , 0.48, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.86, 0.01, 0.14, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.84, 0.16, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.85, 0.15],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

In [162]:
np.around(models['cachly'].transmat_, 2)

array([[0.85, 0.15, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.83, 0.16, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.85, 0.01, 0.14, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.86, 0.  , 0.14, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.76, 0.24, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.85, 0.05, 0.1 , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.83, 0.17, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.9 , 0.09, 0.01, 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.79, 0.21, 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ,

In [163]:
np.around(models['chungta'].transmat_, 2)

array([[0.51, 0.49, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.77, 0.13, 0.1 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.69, 0.  , 0.31, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.52, 0.35, 0.12, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.74, 0.26, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.85, 0.08, 0.07, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.8 , 0.  , 0.2 , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.68, 0.32, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.22, 0.64, 0.14,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ,

In [164]:
np.around(models['nguoi'].transmat_, 2)

array([[0.64, 0.09, 0.27, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.57, 0.01, 0.42, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.7 , 0.12, 0.18, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.74, 0.  , 0.26, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.62, 0.38, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.87, 0.13, 0.01, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.71, 0.26, 0.02],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.65, 0.35],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

## Test dữ liệu nói từ micro

In [175]:
testset1 = {}
n_test1 = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    testset1[cname] = get_class_data(os.path.join('test', cname+'_test'))
    n_test1[cname] = len(testset1[cname])

Load khong dataset
Load tôi dataset
Load cachly dataset
Load chungta dataset
Load nguoi dataset


In [176]:
for cname in class_names:
    testset1[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in testset1[cname]])

In [178]:
print("Testing")
n_correct1 = {'khong': 0, 'tôi': 0, 'chungta': 0, 'nguoi': 0,'cachly': 0}
for true_cname in class_names:
    for O in testset1[true_cname]:
        score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
        if (true_cname == max(score, key=score.get)): n_correct1[true_cname] += 1
        print(true_cname, score, 'predict:', max(score, key=score.get))

Testing
khong {'tôi': -inf, 'khong': -21.16992321721046, 'chungta': -inf, 'nguoi': -inf, 'cachly': -inf} predict: khong
khong {'tôi': -48.93742939310759, 'khong': -33.95479722191512, 'chungta': -36.96217995472938, 'nguoi': -75.61372750588033, 'cachly': -66.0974192405975} predict: khong
khong {'tôi': -121.3959382587892, 'khong': -64.29140635094593, 'chungta': -329.93513416270866, 'nguoi': -57.6312594165811, 'cachly': -56.62426820105277} predict: cachly
khong {'tôi': -41.15884638829622, 'khong': -31.181263272061482, 'chungta': -38.55125973610246, 'nguoi': -43.54860743681232, 'cachly': -43.917562164662264} predict: khong
khong {'tôi': -85.5174917658584, 'khong': -36.378739221159506, 'chungta': -56.499844130510745, 'nguoi': -53.168727082788244, 'cachly': -58.55225647259785} predict: khong
khong {'tôi': -127.42077440221723, 'khong': -45.64086844465595, 'chungta': -65.00611281393837, 'nguoi': -46.16178887787726, 'cachly': -37.63695729147794} predict: cachly
khong {'tôi': -61.20495462430816, 

In [179]:
for cname in class_names:
    print('Accuracy:', cname, n_correct1[cname]/n_test1[cname])

Accuracy: khong 0.8387096774193549
Accuracy: tôi 0.7
Accuracy: cachly 1.0
Accuracy: chungta 1.0
Accuracy: nguoi 0.8214285714285714


In [180]:
print(n_test1)

{'khong': 31, 'tôi': 10, 'cachly': 9, 'chungta': 10, 'nguoi': 28}


In [181]:
np.around(models['nguoi'].transmat_, 2)

array([[0.64, 0.09, 0.27, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.57, 0.01, 0.42, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.7 , 0.12, 0.18, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.74, 0.  , 0.26, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.62, 0.38, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.87, 0.13, 0.01, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.71, 0.26, 0.02],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.65, 0.35],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

In [182]:
import pickle

In [183]:
filename = 'hmm_models.sav'
pickle.dump(models, open(filename, 'wb'))

In [184]:
pickle.dump(kmeans, open('kmeans.sav', 'wb'))